[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SlimeVRX/SlimeVRX/blob/main/hipnuc/document/08.ipynb)

#**Study [Attitude for IMU & AHRS](https://zhuanlan.zhihu.com/p/351596374)**
#**Lesson 08: Summary and Examples** [(3D姿态总结与例子)](https://zhuanlan.zhihu.com/p/101101455)

### So sánh Attitude Array, Quaternion và Euler Angle

Hình sau minh họa rõ ràng sự so sánh giữa ba phương pháp toán học chính để biểu diễn các tư thế 3D. Ma trận Attitude có thể nói là “mẹ của mọi phương pháp”, các góc Euler là trực quan nhất nhưng lại mắc phải vấn đề bế tắc Euler (Điểm kỳ dị), và thứ tự quay cũng gây khó chịu, một "phiên bản nén của ma trận Attitude". Các thuộc tính hoàn toàn giống với mảng Attitude và so với mảng Attitude, nó có ưu điểm là ít dữ liệu hơn và ít tính toán hơn. Thuận tiện nhất cho máy tính chi phí thấp!

<p align="left"><img width="500", src="https://pic4.zhimg.com/80/v2-0941b4c54dd7c79705af5086c841a9d3_720w.jpg"></p>

Hãy làm một bài tập lớn với tất cả các hàm thư viện trước đó:

In [ ]:
import numpy as np
sin = np.sin
cos = np.cos

In [ ]:
# # ------------------------- Exercises1: ------------------------------
def ch_eul2m_321(i):
    alpha = i[2]     # (Alpha Z)
    beta = i[0]      # (Beta X)
    gamma = i[1]     # (Gamma Y)

    s_z_alpha = sin(alpha); s_x_beta = sin(beta); s_y_gamma = sin(gamma)
    c_z_alpha = cos(alpha); c_x_beta = cos(beta); c_y_gamma = cos(gamma)

    Cb2n_321 = np.array([[ c_y_gamma*c_z_alpha, s_x_beta*s_y_gamma*c_z_alpha-c_x_beta*s_z_alpha, c_x_beta*s_y_gamma*c_z_alpha+s_x_beta*s_z_alpha],
                         [c_y_gamma*s_z_alpha, s_x_beta*s_y_gamma*s_z_alpha+c_x_beta*c_z_alpha, c_x_beta*s_y_gamma*s_z_alpha-s_x_beta*c_z_alpha],
                         [-s_y_gamma,    s_x_beta*c_y_gamma,          c_x_beta*c_y_gamma            ]])

    return Cb2n_321

def ch_eul2m_312(i):
    alpha = i[2]     # (Alpha Z)
    beta = i[0]      # (Beta X)
    gamma = i[1]     # (Gamma Y)

    s_z_alpha = sin(alpha); s_x_beta = sin(beta); s_y_gamma = sin(gamma)
    c_z_alpha = cos(alpha); c_x_beta = cos(beta); c_y_gamma = cos(gamma)

    Cb2n_312 = np.array([[ c_y_gamma*c_z_alpha-s_x_beta*s_y_gamma*s_z_alpha, -c_x_beta*s_z_alpha,  s_y_gamma*c_z_alpha+s_x_beta*c_y_gamma*s_z_alpha],
                         [c_y_gamma*s_z_alpha+s_x_beta*s_y_gamma*c_z_alpha,  c_x_beta*c_z_alpha,  s_y_gamma*s_z_alpha-s_x_beta*c_y_gamma*c_z_alpha],
                         [-c_x_beta*s_y_gamma,           s_x_beta,     c_x_beta*c_y_gamma           ]])

    return Cb2n_312

def ch_m2q(Cb2n):
    C11 = Cb2n[0,0]; C12 = Cb2n[0,1]; C13 = Cb2n[0,2]
    C21 = Cb2n[1,0]; C22 = Cb2n[1,1]; C23 = Cb2n[1,2]
    C31 = Cb2n[2,0]; C32 = Cb2n[2,1]; C33 = Cb2n[2,2]

    if C11 >= C22+C33:
        q1 = 0.5*np.sqrt(1+C11-C22-C33)
        q0 = (C32-C23)/(4*q1); q2 = (C12+C21)/(4*q1); q3 = (C13+C31)/(4*q1)
    elif C22>= C11+C33:
        q2 = 0.5*np.sqrt(1-C11+C22-C33)
        q0 = (C13-C31)/(4*q2); q1 = (C12+C21)/(4*q2); q3 = (C23+C32)/(4*q2)
    elif C33>=C11+C22:
        q3 = 0.5*np.sqrt(1-C11-C22+C33)
        q0 = (C21-C12)/(4*q3); q1 = (C13+C31)/(4*q3); q2 = (C23+C32)/(4*q3)
    else:
        q0 = 0.5*np.sqrt(1+C11+C22+C33)
        q1 = (C32-C23)/(4*q0); q2 = (C13-C31)/(4*q0); q3 = (C21-C12)/(4*q0)
    Qb2n = np.array([q0, q1, q2, q3], dtype=np.float64)
    return Qb2n

In [ ]:
eul321 = np.array([-30, 30, 45])
print("Euler angles (321):\n", eul321)

Cb2n = ch_eul2m_321(np.deg2rad(eul321))
print("Euler angle rotation Attitude matrix Cb2n:\n", Cb2n)

Qb2n = ch_m2q(Cb2n)
print("Euler angle to Quaternion:\n", Qb2n)

Euler angles (321):
 [-30  30  45]
Euler angle rotation Attitude matrix Cb2n:
 [[ 0.61237244 -0.78914913 -0.04736717]
 [ 0.61237244  0.43559574  0.65973961]
 [-0.5        -0.4330127   0.75      ]]
Euler angle to Quaternion:
 [ 0.83635641 -0.32664074  0.13529903  0.4189367 ]


In [ ]:
atan2 = np.arctan2
asin = np.arcsin

In [ ]:
# # ------------------------- Exercises2: ------------------------------
def ch_m2eul_321(Cb2n):
    beta = atan2(Cb2n[2,1], Cb2n[2,2])
    alpha = asin(-Cb2n[2,0])
    gamma = atan2(Cb2n[1,0],Cb2n[0,0])
    return beta, alpha, gamma

In [ ]:
Cb2n = np.array([[0.612372, -0.78915, -0.04737],
                 [0.612372, 0.435596, 0.65974],
                 [-0.5, -0.43301, 0.75]], dtype=np.float64)

print("Attitude array Cb2n:\n", Cb2n)

np.set_printoptions(suppress=True)
print("Attitude matrix multiplied by own transpose = unit matrix. Think about why?>~\n", Cb2n.dot(Cb2n.T))

a,b,c = ch_m2eul_321(Cb2n)
print("Attitude array to Euler angle:\n %.3f %.3f %.3f " %(np.rad2deg(a), np.rad2deg(b), np.rad2deg(c)))

Qb2n = ch_m2q(Cb2n)
print("Euler angle to quaternion:\n", Qb2n)

Attitude array Cb2n:
 [[ 0.612372 -0.78915  -0.04737 ]
 [ 0.612372  0.435596  0.65974 ]
 [-0.5      -0.43301   0.75    ]]
Attitude matrix multiplied by own transpose = unit matrix. Think about why?>~
 [[ 1.00000111 -0.000003   -0.00000366]
 [-0.000003    1.00000021  0.00000158]
 [-0.00000366  0.00000158  0.99999766]]
Attitude array to Euler angle:
 -30.000 30.000 45.000 
Euler angle to quaternion:
 [ 0.83635638 -0.32664006  0.13529818  0.41893684]


In [ ]:
# # ------------------------- Exercises3: ------------------------------
def ch_q2m(Qb2n):
    q11 = Qb2n[0]*Qb2n[0]; q12 = Qb2n[0]*Qb2n[1]; q13 = Qb2n[0]*Qb2n[2]; q14 = Qb2n[0]*Qb2n[3]
    q22 = Qb2n[1]*Qb2n[1]; q23 = Qb2n[1]*Qb2n[2]; q24 = Qb2n[1]*Qb2n[3]
    q33 = Qb2n[2]*Qb2n[2]; q34 = Qb2n[2]*Qb2n[3]
    q44 = Qb2n[3]*Qb2n[3]
    Cb2n = np.array([[ q11+q22-q33-q44,  2*(q23-q14),     2*(q24+q13)],
                     [2*(q23+q14),      q11-q22+q33-q44, 2*(q34-q12)],
                     [2*(q24-q13),      2*(q34+q12),     q11-q22-q33+q44 ]], dtype=np.float64)
    return Cb2n

def ch_q2eul_321(Qb2n):
    q0 = Qb2n[0]
    q1 = Qb2n[1]
    q2 = Qb2n[2]
    q3 = Qb2n[3]

    roll =  atan2( 2*( q0*q1 + q2*q3 ) , 1 - 2*q1*q1 - 2*q2*q2)
    pitch = asin( 2*(q0*q2 - q1*q3) )
    yaw = atan2(2*( q0*q3 + q1*q2 ), 1 - 2*q2*q2 - 2*q3*q3)
    return roll, pitch, yaw

In [ ]:
Qb2n = np.array([0.836356, -0.32664, 0.135299, 0.418937])

print("Quaternion Qb2n:\n", Qb2n)

Cb2n = ch_q2m(Qb2n)
print("Qb2n Quaternion to Attitude Array Cb2n\n", Cb2n)

# a,b,c = ch_m2eul_321(Cb2n)
# print("Quaternion to Euler angles:\n %.3f %.3f %.3f " %(np.rad2deg(a), np.rad2deg(b), np.rad2deg(c)))

a,b,c = ch_q2eul_321(Qb2n)
print("Quaternion to Euler angles:\n %.3f %.3f %.3f " %(np.rad2deg(a), np.rad2deg(b), np.rad2deg(c)))

Quaternion Qb2n:
 [ 0.836356 -0.32664   0.135299  0.418937]
Qb2n Quaternion to Attitude Array Cb2n
 [[ 0.61237102 -0.78914908 -0.0473669 ]
 [ 0.61237282  0.43559528  0.65973816]
 [-0.49999942 -0.43301113  0.75000006]]
Quaternion to Euler angles:
 -30.000 30.000 45.000 
Quaternion to Euler angles:
 -30.000 30.000 45.000 
